# ADS 509 Assignment 5.1: Topic Modeling

This notebook holds Assignment 5.1 for Module 5 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In this assignment you will work with a categorical corpus that accompanies `nltk`. You will build the three types of topic models described in Chapter 8 of _Blueprints for Text Analytics using Python_: NMF, LSA, and LDA. You will compare these models to the true categories. 


In [3]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 39.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 42.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
ERROR: pip's dependen

In [5]:
pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.1/493.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 10.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.3/880.3 kB 21.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 38.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 39.4 MB/s eta 0:00:0000:01

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [56]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [12]:
# These libraries may be useful to you

#!pip install pyLDAvis==3.4.1 --user  #You need to restart the Kernel after installation.
# You also need a Python version => 3.9.0
from nltk.corpus import brown

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim_models

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

from spacy.lang.en.stop_words import STOP_WORDS as stopwords
import en_core_web_sm

from collections import Counter, defaultdict

nlp = en_core_web_sm.load()

In [112]:
# add any additional libaries you need here
import nltk
from nltk.corpus import brown
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [13]:
nlp

In [14]:
# This function comes from the BTAP repo.

def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

## Getting to Know the Brown Corpus

Let's spend a bit of time getting to know what's in the Brown corpus, our NLTK example of an "overlapping" corpus.

In [19]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/roger.qiu/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [21]:
brown

<CategorizedTaggedCorpusReader in '/Users/roger.qiu/nltk_data/corpora/brown'>

In [20]:
# categories of articles in Brown corpus
for category in brown.categories() :
    print(f"For {category} we have {len(brown.fileids(categories=category))} articles.")

For adventure we have 29 articles.
For belles_lettres we have 75 articles.
For editorial we have 27 articles.
For fiction we have 29 articles.
For government we have 30 articles.
For hobbies we have 36 articles.
For humor we have 9 articles.
For learned we have 80 articles.
For lore we have 48 articles.
For mystery we have 24 articles.
For news we have 44 articles.
For religion we have 17 articles.
For reviews we have 17 articles.
For romance we have 29 articles.
For science_fiction we have 6 articles.


Let's create a dataframe of the articles in of hobbies, editorial, government, news, and romance.

In [77]:
categories = ['editorial','government','news','romance','hobbies'] 

category_list = []
file_ids = []
texts = []

for category in categories : 
    for file_id in brown.fileids(categories=category) :
        
        # build some lists for a dataframe
        category_list.append(category)
        file_ids.append(file_id)
        
        text = brown.words(fileids=file_id)
        texts.append(" ".join(text))

        
        
df = pd.DataFrame()
df['category'] = category_list
df['id'] = file_ids
df['text'] = texts 

df.shape

(166, 3)

In [78]:
print(df.head())

    category    id                                               text
0  editorial  cb01  Assembly session brought much good The General...
1  editorial  cb02  Must Berlin remain divided ? ? The inference h...
2  editorial  cb03  A good man departs . Goodby , Mr. Sam . Sam Ra...
3  editorial  cb04  A shock wave from Africa Word of Dag Hammarskj...
4  editorial  cb05  Help when needed If the Dominican Republic ach...


In [79]:
# Let's add some helpful columns on the df
df['char_len'] = df['text'].apply(len)
df['word_len'] = df['text'].apply(lambda x: len(x.split()))
print(df.head())

    category    id                                               text  \
0  editorial  cb01  Assembly session brought much good The General...   
1  editorial  cb02  Must Berlin remain divided ? ? The inference h...   
2  editorial  cb03  A good man departs . Goodby , Mr. Sam . Sam Ra...   
3  editorial  cb04  A shock wave from Africa Word of Dag Hammarskj...   
4  editorial  cb05  Help when needed If the Dominican Republic ach...   

   char_len  word_len  
0     12659      2200  
1     12544      2234  
2     11871      2244  
3     12284      2230  
4     12479      2241  


In [57]:
%matplotlib inline
df.groupby('category').agg({'word_len': 'mean'}).plot.bar(figsize=(10,6))

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Now do our TF-IDF and Count vectorizations.

In [59]:
count_text_vectorizer = CountVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(df["text"])
count_text_vectors.shape

/Users/roger.qiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


(166, 4941)

In [63]:
count_text_vectors

<166x4941 sparse matrix of type '<class 'numpy.int64'>'
	with 72541 stored elements in Compressed Sparse Row format>

In [60]:
tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(df['text'])
tfidf_text_vectors.shape

(166, 4941)

In [64]:
tfidf_text_vectors

<166x4941 sparse matrix of type '<class 'numpy.float64'>'
	with 72541 stored elements in Compressed Sparse Row format>

Q: What do the two data frames `count_text_vectors` and `tfidf_text_vectors` hold? 

A: count text vectors will have the count of each token as a value and each record will be a document in "text" and each field will be a token. So it is a large df of counts of how many times a word appears in a "text" value. On the other hand, the tfidf text vector is similiar to this in that all the fields are unique words that have appeared through all the documents. The only difference is that the values are not the frequency of the words but rather a score that tells you how important each word is based on appearances in the document and throughout all documents.

## Fitting a Non-Negative Matrix Factorization Model

In this section the code to fit a five-topic NMF model has already been written. This code comes directly from the [BTAP repo](https://github.com/blueprints-for-text-analytics-python/blueprints-text), which will help you tremendously in the coming sections. 

In [65]:
nmf_text_model = NMF(n_components=5, random_state=314)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)
H_text_matrix = nmf_text_model.components_

/Users/roger.qiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [66]:
display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out())


Topic 00
  mr (0.51)
  president (0.45)
  kennedy (0.43)
  united (0.42)
  khrushchev (0.40)

Topic 01
  said (0.88)
  didn (0.46)
  ll (0.45)
  thought (0.42)
  man (0.37)

Topic 02
  state (0.40)
  development (0.36)
  tax (0.33)
  sales (0.30)
  program (0.25)

Topic 03
  mrs (2.61)
  mr (0.78)
  said (0.64)
  miss (0.52)
  car (0.51)

Topic 04
  game (1.01)
  league (0.74)
  ball (0.72)
  baseball (0.71)
  team (0.66)


Now some work for you to do. Compare the NMF categorization to the original categories from the Brown Corpus.

We are interested in the extent to which our NMF categorization agrees or disagrees with the original categories in the corpus. For each topic in your NMF model, tally the Brown categories and interpret the results. 


In [71]:
# Your code here
# the w text matrix will show us how much each document record is associated with each category
W_text_matrix[:1]

# here we can see the association of the first record with the 5 different categories
# the record has the highest association with the 3rd document (news) so it should be assigned to news.

array([[0.16833745, 0.        , 0.19665029, 0.        , 0.02859793]])

In [69]:
# assign all records in the W test matrix to a category using argmax
topic_assignments = np.argmax(W_text_matrix, axis=1)
topic_assignments

array([2, 0, 0, 0, 0, 0, 0, 1, 1, 4, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 0, 0, 0, 0, 2, 3,
       3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3,
       4, 3, 0, 2, 2, 0, 4, 4, 3, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 3, 2, 2, 2, 1, 4, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [75]:
categories = ['editorial','government','news','romance','hobbies'] 

# now use numpy to map the numeric topic assignments to category names
mapped_categories = np.array(categories)[topic_assignments]
mapped_categories[:5]

array(['news', 'editorial', 'editorial', 'editorial', 'editorial'],
      dtype='<U10')

In [80]:
# join the categorizations with df to compare
df['predicted_category'] = mapped_categories
print(df.head())

    category    id                                               text  \
0  editorial  cb01  Assembly session brought much good The General...   
1  editorial  cb02  Must Berlin remain divided ? ? The inference h...   
2  editorial  cb03  A good man departs . Goodby , Mr. Sam . Sam Ra...   
3  editorial  cb04  A shock wave from Africa Word of Dag Hammarskj...   
4  editorial  cb05  Help when needed If the Dominican Republic ach...   

   char_len  word_len predicted_category  
0     12659      2200               news  
1     12544      2234          editorial  
2     11871      2244          editorial  
3     12284      2230          editorial  
4     12479      2241          editorial  


In [82]:
# use confusion matrix to check how the predicted_category compared to the real category
conf_matrix = confusion_matrix(df['category'], df['predicted_category'])
conf_matrix

array([[20,  4,  1,  2,  0],
       [ 4,  0,  0, 26,  0],
       [ 0,  8,  1, 26,  1],
       [ 8,  0,  8, 11, 17],
       [ 0, 29,  0,  0,  0]])

In [84]:
# create a table to visualize the results
categories = sorted(df['category'].unique())
conf_df = pd.DataFrame(conf_matrix, index=categories, columns=categories)

print(conf_df)

            editorial  government  hobbies  news  romance
editorial          20           4        1     2        0
government          4           0        0    26        0
hobbies             0           8        1    26        1
news                8           0        8    11       17
romance             0          29        0     0        0


In [85]:
overall_accuracy = accuracy_score(df['category'], df['predicted_category'])
overall_accuracy

0.1927710843373494

Q: How does your five-topic NMF model compare to the original Brown categories? 

A: it appears the the five topic model performed rather poorly, it only achieved an accuracy of 19%. It correctly predicted most of the editorial categories but all the others had very poor accuracy, especially government and romance, in which it did not predict any of them accuractly.

## Fitting an LSA Model

In this section, follow the example from the repository and fit an LSA model (called a "TruncatedSVD" in `sklearn`). Again fit a five-topic model and compare it to the actual categories in the Brown corpus. Use the TF-IDF vectors for your fit, as above. 

To be explicit, we are once again interested in the extent to which this LSA factorization agrees or disagrees with the original categories in the corpus. For each topic in your model, tally the Brown categories and interpret the results. 


In [86]:
# Your code here
tfidf_text_vectors[:1]

<1x4941 sparse matrix of type '<class 'numpy.float64'>'
	with 560 stored elements in Compressed Sparse Row format>

In [87]:
# create model useing 5 components
lsa_model = TruncatedSVD(n_components=5, random_state=42)
lsa_model

TruncatedSVD(n_components=5, random_state=42)

In [89]:
# fit the LSA model using 5 topics from the tfidf_text_vectors
lsa_topic_matrix = lsa_model.fit_transform(tfidf_text_vectors)
lsa_topic_matrix[:1]

array([[ 0.38957128, -0.24975864, -0.01892367, -0.05539946, -0.00469051]])

In [90]:
# use argmax again to assign topics to documents
topic_assignments = np.argmax(lsa_topic_matrix, axis=1)
topic_assignments

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 4, 4, 4, 4, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [91]:
# now use numpy to map the numeric topic assignments to category names
mapped_categories = np.array(categories)[topic_assignments]
mapped_categories[:5]

array(['editorial', 'editorial', 'editorial', 'editorial', 'editorial'],
      dtype='<U10')

In [92]:
# join the categorizations with the df
df['predicted_lsa_categories'] = mapped_categories
print(df.head())

    category    id                                               text  \
0  editorial  cb01  Assembly session brought much good The General...   
1  editorial  cb02  Must Berlin remain divided ? ? The inference h...   
2  editorial  cb03  A good man departs . Goodby , Mr. Sam . Sam Ra...   
3  editorial  cb04  A shock wave from Africa Word of Dag Hammarskj...   
4  editorial  cb05  Help when needed If the Dominican Republic ach...   

   char_len  word_len predicted_category predicted_lsa_categories  
0     12659      2200               news                editorial  
1     12544      2234          editorial                editorial  
2     11871      2244          editorial                editorial  
3     12284      2230          editorial                editorial  
4     12479      2241          editorial                editorial  


In [93]:
# use confusion matrix to check how the predicted_category compared to the real category
conf_matrix = confusion_matrix(df['category'], df['predicted_lsa_categories'])
conf_matrix

array([[27,  0,  0,  0,  0],
       [30,  0,  0,  0,  0],
       [36,  0,  0,  0,  0],
       [34,  0,  0,  3,  7],
       [21,  8,  0,  0,  0]])

In [94]:
overall_accuracy = accuracy_score(df['category'], df['predicted_lsa_categories'])
overall_accuracy

0.18072289156626506

Q: How does your five-topic LSA model compare to the original Brown categories? 

A: the LSA models appears to have performed as poorly as the NMF model above , it only achieved an accuracy of 18%. It predicts almost all the categories to be the 1st category with a few exceptions for just 1 category.

In [106]:
feature_names = count_text_vectorizer.get_feature_names_out()
feature_names

array(['00', '000', '10', ..., 'youth', 'zone', 'zoning'], dtype=object)

In [107]:
# call display_topics on your model
display_topics(lsa_model, feature_names)


Topic 00
  said (0.44)
  mr (0.25)
  mrs (0.22)
  state (0.20)
  man (0.17)

Topic 01
  said (3.89)
  ll (2.73)
  didn (2.63)
  thought (2.20)
  got (1.97)

Topic 02
  mrs (3.14)
  mr (1.73)
  said (1.05)
  kennedy (0.81)
  president (0.77)

Topic 03
  mrs (30.38)
  club (6.70)
  game (6.40)
  jr (5.81)
  dallas (5.50)

Topic 04
  game (4.33)
  league (3.09)
  baseball (3.06)
  ball (2.94)
  team (2.81)


Q: What is your interpretation of the display topics output? 

A: 
- So based on the display topics output for each LDA topic, it seems that the topics follow these themes: 
- Topic 0 seems to focus on descriptions of characters such as mr or mrs. 
- Topic 1 is more about grammar terms like said, got, thought. 
- Topic 2 seems to be about presidential related words such as president and kennedy 
- Topic 3 and 4 is more about sports and clubs 

## Fitting an LDA Model

Finally, fit a five-topic LDA model using the count vectors (`count_text_vectors` from above). Display the results using `pyLDAvis.display` and describe what you learn from that visualization. 

In [98]:
# Fit your LDA model here
lda_text_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_topic_matrix = lda_text_model.fit_transform(count_text_vectors)
lda_topic_matrix[:1]

array([[2.53132081e-04, 2.40930942e-01, 7.49184801e-01, 2.52364936e-04,
        9.37875996e-03]])

In [104]:
feature_names = count_text_vectorizer.get_feature_names_out()
feature_names

array(['00', '000', '10', ..., 'youth', 'zone', 'zoning'], dtype=object)

In [113]:
# Call `display_topics` on your fitted model here
display_topics(lda_text_model, feature_names)


Topic 00
  home (0.46)
  right (0.35)
  game (0.35)
  way (0.34)
  set (0.34)

Topic 01
  state (0.89)
  development (0.57)
  states (0.51)
  program (0.46)
  use (0.44)

Topic 02
  said (0.93)
  mr (0.73)
  president (0.63)
  state (0.49)
  mrs (0.48)

Topic 03
  feed (0.90)
  general (0.73)
  business (0.70)
  property (0.69)
  shall (0.67)

Topic 04
  said (1.56)
  man (0.62)
  little (0.62)
  old (0.59)
  good (0.53)


In [114]:
# use argmax again to assign topics to documents
topic_assignments = np.argmax(lda_topic_matrix, axis=1)
topic_assignments

array([2, 2, 2, 2, 2, 2, 2, 4, 2, 0, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 3, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 3, 2, 1, 3, 3, 3,
       3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 1, 3, 2, 0, 0, 2, 2, 2, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4,
       2, 1, 0, 1, 0, 3, 4, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 3, 3, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [115]:
# now use numpy to map the numeric topic assignments to category names
mapped_categories = np.array(categories)[topic_assignments]
mapped_categories[:5]

array(['hobbies', 'hobbies', 'hobbies', 'hobbies', 'hobbies'],
      dtype='<U10')

In [116]:
# join the categorizations with the df
df['predicted_lda_categories'] = mapped_categories
print(df.head())

    category    id                                               text  \
0  editorial  cb01  Assembly session brought much good The General...   
1  editorial  cb02  Must Berlin remain divided ? ? The inference h...   
2  editorial  cb03  A good man departs . Goodby , Mr. Sam . Sam Ra...   
3  editorial  cb04  A shock wave from Africa Word of Dag Hammarskj...   
4  editorial  cb05  Help when needed If the Dominican Republic ach...   

   char_len  word_len predicted_category predicted_lsa_categories  \
0     12659      2200               news                editorial   
1     12544      2234          editorial                editorial   
2     11871      2244          editorial                editorial   
3     12284      2230          editorial                editorial   
4     12479      2241          editorial                editorial   

  predicted_lda_categories  
0                  hobbies  
1                  hobbies  
2                  hobbies  
3                  hobbies  
4

In [117]:
# use confusion matrix to check how the predicted_category compared to the real category
conf_matrix = confusion_matrix(df['category'], df['predicted_lda_categories'])
conf_matrix

array([[ 3,  1, 22,  0,  1],
       [ 0, 20,  2,  8,  0],
       [16, 14,  1,  3,  2],
       [ 9,  3, 31,  1,  0],
       [ 0,  0,  0,  0, 29]])

In [118]:
overall_accuracy = accuracy_score(df['category'], df['predicted_lda_categories'])
overall_accuracy

0.3253012048192771

Q: What inference do you draw from the displayed topics for your LDA model? 

A:
Based on the displayed topics, there doesn't seem to be too many similiarities between how the different words are grouped. It is difficult to draw inferences from this.

Q: Repeat the tallying of Brown categories within your topics. How does your five-topic LDA model compare to the original Brown categories? 

A: the LDA model appears to have performed slightly better than the other 2 models , it achieved an accuracy of 32.5%. It predicts mostly the 5th cateogry correctly, the 2nd category is also mostly correct but struggles with the rest of the categories.

In [109]:
lda_display = pyLDAvis.lda_model.prepare(lda_text_model, count_text_vectors, count_text_vectorizer, sort_topics=False)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [100]:
pyLDAvis.display(lda_display)

NameError: name 'lda_display' is not defined

Q: What conclusions do you draw from the visualization above? Please address the principal component scatterplot and the salient terms graph.

A: <!-- Your answer here --> 
